# Introduction to metaprogramming: "Code that creates code" 

Julia has strong **metaprogramming** capabilities. What does this mean?

> **meta**: something on a higher level

**metaprogramming** = "higher-level programming"

i.e. writing code (a program) to manipulate not data, but code (that itself manipulates data)


## Motivating example: Interact.jl

#### Exercise 1

1. Install the `Interact.jl` package.


2. Run the following code

In [9]:
using Interact

@manipulate for i in 1:10
    j = i^2
    "The square of $i is $(j)"
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope(Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["i"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")))], Dict{Symbol,Any}(:className=>"interact-flex-row-left")), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>10,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()))], Dict{Symbol,Any}(:className=>"interact-flex-row-center")), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")))], Dict{Symbol,Any}(:className=>"interact-flex-row-right"))], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Int64} with 2 listeners. Value:
5, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/dpsanders/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/dpsanders/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/dpsanders/.julia/packages/InteractBase/l8cEC/src/../assets/all.js"), Asset("css", nothing, "/Users/dpsanders/.julia/packages/InteractBase/l8cEC/src/../assets/style.css"), Asset("css", nothing, "/Users/dpsanders/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"1265801823703451066\",\"id\":\"ob_10\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"1265801823703451066\",\"id\":\"ob_09\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArra

You should see a slider appear with the label `i`. When you manipulate the slider, the caption should show the value of `i` and its square, and should update when you move the slider.

What happened here? A `for` loop iterates over its values. Somehow the `@manipulate` command, which is a **macro**, took the object "code for the `for` loop" and replaced it with "code for a slider with the same range"; in other words, `@manipulate` operated on code to produce some different code that did something useful -- it took in a program and replaced it by a different program.

We can see the result of the `@manipulate` by "expanding" the effect of the macro using `@macroexpand`. To make it easier to read, we will suppress some line information using the `MacroTools` package:

In [10]:
code = @macroexpand @manipulate for i in 1:10
    j = i^2
    "The square of $i is $j"
end;

using MacroTools
MacroTools.striplines(code)

quote
    local #88#children = (Widgets.OrderedDict)(:i => (i = (Widgets.widget)(1:10, label="i")))
    local #89#output = begin
                #90###370 = ((i,)->begin
                            j = i ^ 2
                            "The square of $(i) is $(j)"
                        end)
                (Widgets.map)(#90###370, i)
            end
    local #91#layout = (Widgets.manipulatelayout)((Widgets.get_backend)())
    (Widgets.Widget){:manipulate}(#88#children, output=#89#output, layout=#91#layout)
end

We see that the `for` loop has been replaced by code for manipulating a widget. The information about the variable name `i`,  the range `1:10` and and the code inside the `for` loop have been preserved, but they have been embedded in a certain way into a new *piece of code*.

In order to carry out a *code transformation* like this, Julia allows us to *manipulate Julia code from within Julia*: we need to be able to get inside a piece of Julia code and modify it, *before* that code reaches the Julia compiler.

Finally we need to see how to wrap the result up into a macro. 

## Expressions 

Let's start with just the part `j = i^2`. If we type this code into a *fresh* Julia session, we get the following error:
```
julia> j  = i^2
ERROR: UndefVarError: i not defined
Stacktrace:
 [1] top-level scope at none:0
```
since Julia is trying to *evaluate* the code using the values for the variables `i` and `j`, which are not defined.

[If instead we type this after running the above `@manipulate` command, `i` is interpreted as a slider and we get a different error.]

For metaprogramming purposes, we do not wish to *evaluate* the code; instead, we just want to treat the code as unevaluated symbolic expressions, which will gain meaning only later. Julia allows us to construct unevaluated pieces of code as follows:

In [14]:
quote 
    j = i^2
end

quote
    #= In[14]:2 =#
    j = i ^ 2
end

or with the following shorthand syntax:

In [16]:
:(j = i^2)

:(j = i ^ 2)

#### Exercise 2

1. Define a variable `code` to be `:(j = i^2)`. 


2. What type is the object `code`? Note that `code` is just a normal Julia variable, of a particular special type.


3. Use the `dump` function to see what there is inside `code`. 
Remembering that `code` is just a particular kind of Julia object, use the Julia to play around interactively, seeing how you can extract pieces of the `code` object.


4. How is the operation `i^2` represented? What kind of object is that subpiece?


5. Copy `code` into a variable `code2`. *Modify* this to replace the power `2` with a power `3`. Make sure that the original `code` variable is *not* also modified.


6. Copy `code2` to a variable `code3`. Replace `i` with `i + 1` in `code3`.


7. Define a variable `i` with the value `4`. *Evaluate* the different `code` expressions using the `eval` function and check the *value* of the variable `j`.


We have just taken a (very simple) program, represented by `code`, and produced two new programs, `code2` and `code3`, which we ran using `eval`. This is the basis of all metaprogramming: taking in a piece of code, and modifying it to produce a new piece of code.

## Walking a syntax tree 

In the previous exercise, we modified a single `i` to `i + 1`. But in general we might have a more complicated expression like `i^2 + (i * (i - 3))` and we may wish to modify *all* of the `i`s in the expression to `(i+1)`s or to `k`s to produce the new expression. The problem is that they may be buried arbitrarily deeply. We thus need to find a way of walking through the whole expression to examine each subpiece of it. 

#### Exercise 3 

1. Write a function `walk!` that takes an expression object and replaces **all** of the `:x`s by `:z`s. 

    Hint: This function should be *recursive*: at some point it will need to call itself if it finds that a piece of the expression is itself an `Expr`.
    
    
2. Make this function into a more general pattern matcher that looks for a given sub-expression and replaces it by another.

#### Exercise 4
Julia by default uses standard 64-bit (or 32-bit) integers, which leads to surprising overflow behaviour, e.g.

In [76]:
2^32 * 2^31

-9223372036854775808

No warning is given that there was an overflow in this calculation. 

However, in `Base` there are *checked* operations, such as `checked_mul`, which do throw an exception on overflow:

In [41]:
Base.checked_mul(2^60, 2^60)

OverflowError: OverflowError: 1152921504606846976 *y overflowed for type Int64

1. Write a function `make_checked` that replaces standard functions (`-`, `+`, `*`, `/`) in an expression by their corresponding checked counterparts.

## Generating repetitive code

One common application of basic metaprogramming in Julia is generating repetitive code. For example, there are situations in which it's useful to wrap on object of one type into a user-defined type in order to modify its behaviour in some way. 

e.g. Let's define a wrapper type `MyFloat` of `Float64`:

In [42]:
struct MyFloat
    x::Float64
end

We can generate objects of this type:

In [43]:
a = MyFloat(3)
b = MyFloat(4)

MyFloat(4.0)

But arithmetic operations are not defined:

In [44]:
a + b

MethodError: MethodError: no method matching +(::MyFloat, ::MyFloat)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

We can define them in the natural way:

In [45]:
import Base: +, -, *, /

+(a::MyFloat, b::MyFloat) = a.x + b.x
-(a::MyFloat, b::MyFloat) = a.x - b.x

- (generic function with 188 methods)

But this will quickly get dull, and we could easily make a mistake. 
As usual, whenever we are repeating something more than twice, we should try to automate it.

We have some code of the form

    op(a::MyFloat, b::MyFloat) = op(a.x, b.x)
    
where `op` denotes the operator. Julia allows us to do this almost literally; we just need to substitute in the *value* of the *variable* `op`! 

#### Exercise 5

1. Let `op` be the symbol `:+`, which is an unevaluated version of the `+` operator.


2. Let `code` be the expression corresponding to `op(a::MyFloat, b::MyFloat) = op(a.x, b.x)`.


3. Substitute the *value* of `op` by replacing `op` by `$(op)`. Check that `code` contains the correct result.


4. Evaluate the code in order to generate the new method. Check that the method works for objects of type `MyFloat`.


5. We can replace the two steps "define `code`" and "evaluate `code`" by one step, `@eval` of the expression that defines `code`.


6. Write a loop over the operations `+`, `-`, `*` and `/` to define them all for our wrapper type.

Finally we need to evaluate the code. The combination of `eval` and `:(...)` that we used above can be abbreviated to `@eval`:

## Macros

Finally let's return to macros. Recall that macros begin with `@` and behave like "super-functions", which take in a piece of code and replace it with another piece of code.
In fact, the effect of a macro call will be to insert the new piece of code in place of the old code, which is consequently compiled by the Julia compiler. 

Note that the user *does not need to explicitly pass an `Expr`ession object*; Julia turns the code that follows the macro call into an expression.

To see this, let's define the simplest macro:

In [1]:
macro simple(expr)
    @show expr
    nothing   # return nothing for the moment
end

@simple (macro with 1 method)

and run it with the following simple code:

In [2]:
@simple yy = xx^2

expr = :(yy = xx ^ 2)


We see that the Julia code that follows the macro call is passed to the macro, *already having been parsed into an `Expr` object*.

#### Exercise 6

1. Define a macro `@simple2` that returns the expression that was passed to it.


2. What happens when you call `@simple2 yy = xx^2`?


3. Define a variable `xx` with the value `3`. Does the macro work now?


4. Does the variable `xx` now exist?


5. To see what's happening, use `@macroexpand`.

You should find that the variable `yy` does *not* now exist, even though it seems like it should, since the code `yy = xx^2` was evaluated. However, macros by default do not "touch" variables in the context from where they are called, since this may have unintended consequences. We refer to macro **hygiene** (they do not "infect" code where they are not welcome).

Nonetheless, often we may *wish* them to modify variables in the context from which they are called, in which case we can "escape" from this hygiene using `esc`:

In [10]:
macro simple3(expr)
    return :($(esc(expr)))
end

@simple3 (macro with 1 method)

Note that once again the macro must return an *expression*. 

#### Exercise 7

1. Check that `@simple3` does create a variable `yy`.

When writing macros, it is common to treat the macro as simply a wrapper around a function that does the hard work of transforming one `Expr` into another`:

#### Exercise 8

1. Write a macro `@walk!` that uses the function `walk!` defined above to replace terms in an expression. Apply it to `yy = xx^2`, replacing `xx` by `xx + 1`.


2. Write a macro `@checked` that replaces all arithmetic operations with checked operations.